In [45]:
import requests
import json
import random
import datetime

DATABASE_URLS = {
    0: "https://lost-items-f914d-default-rtdb.firebaseio.com/items",
    1: "https://found-items-7dec3-default-rtdb.firebaseio.com/items",
    2: "https://resolved-items-default-rtdb.firebaseio.com/items"
    }

def randomChar(l):
    dicR = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
            "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
            "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    word = ""
    for i in range(l):
        word = word + random.choice(dicR)
    return word

def generate_id(status):
    # Determin prefix
    prefix = 'L' if status.lower() == 'lost' else 'F'
    ID_body = randomChar(10)
    itemID = f"{prefix}{ID_body}"
    return itemID

def go_through_DB(DB: str = "All"):
    DB = DB.lower()
    if DB == "lost":
        response = requests.get(f'{DATABASE_URLS[0]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "found":
        response = requests.get(f'{DATABASE_URLS[1]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "completed":
        response = requests.get(f'{DATABASE_URLS[2]}.json')
        item_dict = response.json()
        for key, value in item_dict.items():
            print(f"Item ID: {key}")
            value["image"] = "Placeholder..."
            print(json.dumps(value, indent=2))
    elif DB == "all":
        for i in [0, 1, 2]:
            print(f"------------{i}------------")
            response = requests.get(f'{DATABASE_URLS[i]}.json')
            item_dict = response.json()
            for key, value in item_dict.items():
                print(f"Item ID: {key}")
                value["image"] = "Placeholder..."
                print(json.dumps(value, indent=2))
    else:
        print("Please set the DB to display. (All, Lost, Found, Completed)")

def add_edit(item_json:dict, item_ID:str = None):
    if not item_ID:
        item_ID = generate_id(item_json["status"])
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        data = json.dumps(item_json)
        url = f"{url_base}/{item_ID}.json"
        response = requests.patch(url, data=data)
        if response.status_code == 200:
            print("Add successfull!!")
        else:
            print("Add unsuccessful!!")
    elif item_ID and len(item_ID) == 11 and item_ID[0] in ["L", "F"]:
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        item_json.pop("image")
        item_json.pop("latitude")
        item_json.pop("longitude")
        data = json.dumps(item_json)
        url = f"{url_base}/{item_ID}.json"
        response = requests.patch(url, data=data)
        if response.status_code == 200:
            print("Edit successfull!!")
        else:
            print("Edit unsuccessful!!")
    else:
        print("Wrong ID or JSON")
        
def delete(item_ID:str):
    for i in [0, 1, 2]:
        print(f"---------Database {i}------------")
        response = requests.delete(f'{DATABASE_URLS[i]}/{item_ID}.json')

def mark_as_resolved(item_ID:str):
    if item_ID and len(item_ID) == 11 and item_ID[0] in ["L", "F"]:
        if item_ID.startswith("L"):
            url_base = DATABASE_URLS[0]
        else:
            url_base = DATABASE_URLS[1]
        response = requests.get(f"{url_base}/{item_ID}.json")
        info = response.json()
        info["completed"] = True
        responseD = requests.delete(f"{url_base}/{item_ID}.json")
        responseA = requests.patch(f"{DATABASE_URLS[2]}/{item_ID}.json", data=json.dumps(info))


In [47]:
go_through_DB("completed")

Item ID: L2lSO2yAAX8
{
  "color": "Red",
  "completed": true,
  "date": 1713309837.860914,
  "description": "A red DE",
  "email": "321",
  "image": "Placeholder...",
  "item_type": "Electronic Devices",
  "latitude": 34.02556114475524,
  "longitude": -118.28481674194337,
  "phone": "123",
  "status": "Lost"
}
Item ID: LNGVxGUwHXD
{
  "color": "Pink",
  "completed": true,
  "date": 1712875087.700651,
  "description": "A Skateboard",
  "email": "board@example.com",
  "image": "Placeholder...",
  "item_type": "Sporting Equipment",
  "latitude": 34.02161303565095,
  "longitude": -118.28327715396883,
  "phone": "2130000555",
  "status": "Lost"
}
Item ID: LWPOsOepA2f
{
  "color": "Yellow",
  "completed": false,
  "date": 1712900256.722552,
  "description": "Manager Test",
  "email": "123423545",
  "image": "Placeholder...",
  "item_type": "Sporting Equipment",
  "latitude": 34.021950944067456,
  "longitude": -118.28788518905641,
  "phone": "925888964321",
  "status": "Lost"
}
Item ID: LzjGN

In [28]:
color = "Purple"
year = 2024
month = 4
day = 16
description = "A Manager Add Test"
email = "Manager@example.com"
item_type = "Scooter"
phone = "111111100000000"
status = "Lost"
date = datetime.datetime(year, month, day)
timestamp = date.timestamp()

item_json = {
  "color": color,
  "completed": False,
  "date": timestamp,
  "description": description,
  "email": email,
  "image": "",
  "item_type": item_type,
  "latitude": 0,
  "longitude": 0,
  "phone": phone,
  "status": status
}

#add_edit(item_json=item_json)

item_ID = "LXPoN6AmNdr"
add_edit(item_json=item_json, item_ID=item_ID)

Edit successfull!!


In [41]:
delete("L2kFe1tc9Vo")

---------Database 0------------
---------Database 1------------
---------Database 2------------


In [46]:
mark_as_resolved("LWPOsOepA2f")